# Analysis around bus stop

### Neccessary libraries to import

In [ ]:
import folium
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from geopandas import GeoDataFrame
from shapely.geometry import Point
from folium.plugins import HeatMap
from geopandas import GeoDataFrame
from pyproj import CRS, Transformer
from typing import Dict, List, Tuple
from geopy.distance import distance as geopy_distance

### Neccesary functions to define

In [ ]:
def convert_bng_to_latlon(chunk: pd.DataFrame) -> pd.DataFrame:
    """
    Converts British National Grid coordinates to latitude and longitude using WGS84 CRS.

    Args:
        chunk: DataFrame containing 'Location_Easting' and 'Location_Northing' columns.

    Returns:
        DataFrame: The input DataFrame with additional 'Latitude' and 'Longitude' columns.
    """
    bng_crs = CRS.from_string('EPSG:27700')  # British National Grid CRS
    wgs84_crs = CRS.from_string('EPSG:4326')  # WGS84 CRS (latitude, longitude)

    transformer = Transformer.from_crs(bng_crs, wgs84_crs, always_xy=True)

    # Apply the coordinate transformation to each row of the chunk DataFrame
    chunk[['Longitude', 'Latitude']] = chunk.apply(
        lambda row: pd.Series(transformer.transform(row['Location_Easting'], row['Location_Northing'])),
        axis=1
    )

    return chunk

In [ ]:
def assign_bus_stops_to_lsoa(bus_df: pd.DataFrame, Barnet_df: pd.DataFrame, london_lsoa: GeoDataFrame) -> Dict[str, List[int]]:
    """Assigns bus stops to corresponding LSOAs (Lower Layer Super Output Areas) based on their geographic coordinates.

    Args:
        bus_df (DataFrame): DataFrame containing bus stop data with columns 'Latitude' and 'Longitude'.
        Barnet_df (DataFrame): DataFrame containing LSOA names.
        london_lsoa (GeoDataFrame): GeoDataFrame containing LSOA boundaries.

    Returns:
        Dict[str, List[int]]: A dictionary mapping LSOA names to a list of corresponding bus stop indices.
    """
    
    # Get the unique LSOA names from Barnet_df
    Barnet_LSOA = Barnet_df['LSOA name'].unique()

    # Create a dictionary to store the LSOA names and the corresponding bus stops
    lsoa_bus_stops = {}

    print(f'There are {len(Barnet_LSOA)} LSOAs to be processed.')
    
    counter = 0
    # Iterate over each LSOA name
    for lsoa_name in Barnet_LSOA:
        # Filter the LSOA data based on the specified name
        filtered_lsoa = london_lsoa[london_lsoa['lsoa11nm'] == lsoa_name]
        
        # Iterate over each row in the bus_df DataFrame
        for index, row in bus_df.iterrows():
            # Get the easting and northing coordinates
            northing = row['Latitude']
            easting = row['Longitude']
            
            # Check if the bus stop falls within the filtered LSOA boundaries
            point = Point(easting, northing)
            if filtered_lsoa.geometry.contains(point).any():
                # Add the LSOA name as an attribute to the bus stop
                bus_df.at[index, 'LSOA name'] = lsoa_name
                # Add the bus stop to the dictionary of LSOA names and bus stops
                if lsoa_name in lsoa_bus_stops:
                    lsoa_bus_stops[lsoa_name].append(index)
                else:
                    lsoa_bus_stops[lsoa_name] = [index]
        
        # Remove the checked LSOA name from further iterations
        if lsoa_name in lsoa_bus_stops:
            for stop_index in lsoa_bus_stops[lsoa_name]:
                bus_df.at[stop_index, 'Checked'] = True

        counter += 1
        print(f'{counter}. {lsoa_name} was processed.')

    # Display the dictionary of LSOA names and the corresponding bus stops
    return lsoa_bus_stops

In [ ]:
def count_burglaries_within_distance(bus_location_df: pd.DataFrame, crime_df: pd.DataFrame, distance: float) -> pd.DataFrame:
    """
    Count the number of burglaries within a specified distance of each bus location.

    Args:
        bus_location_df: DataFrame containing bus locations with latitude and longitude columns.
        crime_df: DataFrame containing crime locations with latitude and longitude columns.
        distance: Distance in meters specifying the range for counting burglaries.

    Returns:
        The original bus location DataFrame with an additional column for burglary counts.

    """

    # Get the latitude and longitude values as arrays
    bus_locations = bus_location_df[['Latitude', 'Longitude']].values
    
    # Add a new column with the burglary counts to the bus location dataframe
    bus_location_df[f"{distance}_meter_away"] = 0
    
    # Iterate over each bus location
    for i, bus_location in enumerate(bus_locations):
        num_burglaries = 0
        # Iterate over each crime location
        for _, crime_row in crime_df.iterrows():
            crime_location = (crime_row['Latitude'], crime_row['Longitude'])
            
            # Calculate the distance between the bus and crime location
            dist = geopy_distance(bus_location, crime_location).meters
            
            # Increment the count if the distance is within the specified range
            if dist <= distance:
                num_burglaries += 1
        
        # Set the burglary count for the current bus location
        bus_location_df.at[i, f"{distance}_meter_away"] = num_burglaries
        print(f'{i} bus stops have been proceed.')

    return bus_location_df


In [ ]:
def calculate_distance_to_nearest_bus(filtered_counts: pd.DataFrame, bus_locations: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the distance to the nearest bus stop for each location in filtered_counts DataFrame.

    Args:
        filtered_counts (pd.DataFrame): DataFrame containing locations to calculate distances for.
        bus_locations (pd.DataFrame): DataFrame containing bus stop locations.

    Returns:
        pd.DataFrame: DataFrame with added columns for distance to the nearest bus stop,
                      bus stop longitude, and bus stop latitude.
    """

    bus_locations = bus_locations[['Latitude', 'Longitude']]
    tree = KDTree(bus_locations)

    distances, indices = tree.query(filtered_counts[['Latitude', 'Longitude']], k=1)

    nearest_bus_stops = bus_locations.iloc[indices.flatten()]

    filtered_counts['distance to bus'] = [
        geopy.distance.distance((lat1, lon1), (lat2, lon2)).meters
        for (lat1, lon1), (lat2, lon2) in zip(
            filtered_counts[['Latitude', 'Longitude']].values,
            nearest_bus_stops.values
        )
    ]

    filtered_counts['bus_long'] = nearest_bus_stops['Longitude'].values
    filtered_counts['bus_lat'] = nearest_bus_stops['Latitude'].values

    return filtered_counts

In [ ]:
def count_burglaries_within_distance(bus_location_df, crime_df, distance):
    # Get the latitude and longitude values as arrays
    bus_locations = bus_location_df[['Latitude', 'Longitude']].values
    
    # Add a new column with the burglary counts to the bus location dataframe
    bus_location_df[f"{distance}_meter_away"] = 0
    
    print(f'{len(bus_locations)} Bus station will be proccessed.')
    # Iterate over each bus location
    for i, bus_location in enumerate(bus_locations):
        num_burglaries = 0
    
        # Iterate over each crime location
        for _, crime_row in crime_df.iterrows():
            crime_location = (crime_row['Latitude'], crime_row['Longitude'])
            
            # Calculate the distance between the bus and crime location
            dist = geopy_distance(bus_location, crime_location).meters
            
            # Increment the count if the distance is within the specified range
            if dist <= distance:
                num_burglaries += 1
        
        # Set the burglary count for the current bus location
        bus_location_df.at[i, f"{distance}_meter_away"] = num_burglaries
        print(f'{i} Bus station has been proccessed.')
    return bus_location_df


In [ ]:
def count_burglaries_within_distance(bus_location_df: pd.DataFrame, crime_df: pd.DataFrame, distance: int) -> Tuple[pd.DataFrame, int]:
    """
    Counts the number of crimes within a specified distance from each bus stop.

    Args:
        bus_location_df (pd.DataFrame): DataFrame containing bus stop locations.
        crime_df (pd.DataFrame): DataFrame containing crime locations.
        distance (int): Distance in meters to consider for counting crimes.

    Returns:
        Tuple[pd.DataFrame, int]: A tuple containing the modified bus location DataFrame with added columns for
                                  the number of crimes within the specified distance, and the number of unique crimes.
    """

    # Get the latitude and longitude values as arrays
    bus_locations = bus_location_df[['Latitude', 'Longitude']].values

    # Add a new column with the burglary counts to the bus location DataFrame
    bus_location_df[f"{distance}_meter_away"] = 0

    unique_crimes = set()  # Set to store unique crimes

    print(f'{len(bus_locations)} bus stations will be processed.')

    # Iterate over each bus location
    for i, bus_location in enumerate(bus_locations):
        num_burglaries = 0

        # Set to store unique crimes for the current bus location
        unique_crimes_for_bus = set()

        # Iterate over each crime location
        for crime_index in crime_df.index:
            crime_row = crime_df.loc[crime_index]
            crime_location = (crime_row['Latitude'], crime_row['Longitude'])

            # Calculate the distance between the bus and crime location
            dist = geopy.distance.distance(bus_location, crime_location).meters

            # Increment the count if the distance is within the specified range
            if dist <= distance:
                num_burglaries += 1
                unique_crimes_for_bus.add(crime_index)

        # Set the burglary count for the current bus location
        bus_location_df.at[i, f"{distance}_meter_away"] = num_burglaries

        # Update the set of unique crimes
        unique_crimes.update(unique_crimes_for_bus)

        print(f'{i+1} bus station has been processed.')

    return bus_location_df, len(unique_crimes)

### Cleaning up the bus stop dataframe

In [ ]:
# Read the bus stops data from the CSV file into a DataFrame
bus_df = pd.read_csv("Data/Bus station analysis/bus-stops-10-06-15.csv")

In [ ]:
# Filter out rows with NaN values in the 'Location_Easting' or 'Location_Northing' columns
bus_df = bus_df.dropna(subset=['Location_Easting', 'Location_Northing']).copy()

In [ ]:
chunk_size: int = 10000
total_rows: int = len(bus_df)
processed_rows: int = 0

# Process DataFrame in chunks
for i in range(0, total_rows, chunk_size):
    # Extract a chunk of data
    chunk = bus_df.iloc[i:i+chunk_size].copy()
    
    # Convert British National Grid coordinates to latitude and longitude
    chunk = convert_bng_to_latlon(chunk)
    
    # Update processed rows count
    processed_rows += len(chunk)
    print(f"Processed {processed_rows} out of {total_rows} rows")
        
    # Update the original DataFrame with the chunk results
    bus_df.loc[i:i+chunk_size, ['Latitude', 'Longitude']] = chunk[['Latitude', 'Longitude']]

# Display the updated DataFrame
bus_df.head()

In [ ]:
columns_to_drop = ['Location_Easting', 'Location_Northing', 'Heading', 'Virtual_Bus_Stop', 'Naptan_Atco', 'Bus_Stop_Code', 'Stop_Code_LBSL']

# Drop the specified columns from the DataFrame
bus_df = bus_df.drop(columns=columns_to_drop)
bus_df

### Creating some visualization about bus stops

In [ ]:
# Create a map centered around London
LondonMapWithBus = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Iterate over each row in the dataframe
for index, row in bus_df.head(150).iterrows():
    # Get the latitude and longitude coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Create a marker for each bus stop
    bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
    bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                    popup=row['Stop_Name'],
                                    icon=bicycle_icon)
    
    # Add the marker to the map
    bus_stop_marker.add_to(LondonMapWithBus)

# Display the map
LondonMapWithBus

In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
# LondonMapWithBus.save("Pictures/Bus station analysis/LondonMapWithBus.html")

In [ ]:
# Load the LSOA boundary data for London
london_lsoa = gpd.read_file('London.geojson')

# Filter the LSOA data for Barnet
barnet_lsoa = london_lsoa[london_lsoa['lsoa11nm'].str.contains('Barnet')]

In [ ]:
# Create a map centered around London
BarnetMapWithBus = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Iterate over each row in the dataframe
for index, row in bus_df.iterrows():
    # Get the latitude and longitude coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Check if the bus stop falls within Barnet LSOA boundaries
    point = Point(longitude, latitude)
    if barnet_lsoa.geometry.contains(point).any():
        # Create a marker for each bus stop
        bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
        bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                        popup=row['Stop_Name'],
                                        icon=bicycle_icon)
        
        # Add the marker to the map
        bus_stop_marker.add_to(BarnetMapWithBus)

# Display the map
BarnetMapWithBus

In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
# BarnetMapWithBus.save("Pictures/Bus station analysis/BarnetMapWithBus.html")

In [ ]:
# Define the list of desired LSOA(hendon and west hendon)
desired_LSOA = ['Barnet 031A', 'Barnet 031B', 'Barnet 031C', 'Barnet 032A', 'Barnet 032B', 'Barnet 032C',
              'Barnet 032D', 'Barnet 032E', 'Barnet 034A', 'Barnet 034B', 'Barnet 034D', 'Barnet 031D',
              'Barnet 032F', 'Barnet 034C', 'Barnet 039D', 'Barnet 036B', 'Barnet 036C', 'Barnet 036D',
              'Barnet 036F']

In [ ]:
# Filter the LSOA data based on the specified names
filtered_lsoa = london_lsoa[london_lsoa['lsoa11nm'].isin(desired_LSOA)]

# Create a map centered around London
DesiredWardsMap = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Iterate over each row in the dataframe
for index, row in bus_df.iterrows():
    # Get the easting and northing coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Check if the bus stop falls within the filtered LSOA boundaries
    point = Point(longitude, latitude)
    if filtered_lsoa.geometry.contains(point).any():
        # Create a marker for each bus stop
        bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
        bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                        popup=row['Stop_Name'],
                                        icon=bicycle_icon)
        
        # Add the marker to the map
        bus_stop_marker.add_to(DesiredWardsMap)

# Display the map
DesiredWardsMap

In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
DesiredWardsMap.save("Pictures/Bus station analysis/DesiredWardsMap.html")

### Adding the LSOA name attribute to the each bus stop

In [ ]:
# Read the data from the Parquet file 
df_burg = pd.read_parquet("burglary.parquet")

# Drop rows with NaN values in "Longitude" and "Latitude" columns
df_burg.dropna(subset=['Longitude', 'Latitude'], inplace=True)

# Reset the index of the DataFrame after dropping rows
df_burg.reset_index(drop=True, inplace=True)

# Create a new DataFrame with rows containing 'Barnet' in LSOA name
Barnet_df = df_burg.loc[df_burg['LSOA name'].str.contains('Barnet')]

# Reset the index of the new DataFrame
Barnet_df.reset_index(drop=True, inplace=True)

# Get the count and unique values of LSOA names in the Barnet DataFrame
lsoa_count = len(Barnet_df['LSOA name'].unique())
Barnet_lsoa_names = Barnet_df['LSOA name'].unique()

lsoa_count, Barnet_lsoa_names

In [ ]:
lsoa_bus_stops = assign_bus_stops_to_lsoa(bus_df, Barnet_df, london_lsoa)

In [ ]:
# Iterate over the dictionary of LSOA names and bus stops
for lsoa_name, bus_stops in lsoa_bus_stops.items():
    # Add the LSOA name to the 'LSOA name' column for the corresponding bus stops
    bus_df.loc[bus_stops, 'LSOA name'] = lsoa_name

# Display the updated bus_df DataFrame
Barnet_bus = bus_df[bus_df['LSOA name'].isnull() == False].copy()
Barnet_bus

### Analyzing the relation between bus stops and the crime hot spots

In [ ]:
# Extract the data for burglary in the desired area(Hendon and west Hendon)
desired_burg = df_burg[df_burg['LSOA name'].isin(desired_LSOA)]
# Count the number of burglarys by the location
coor_burg = desired_burg[['Latitude', 'Longitude']].value_counts().rename_axis(['Latitude', 'Longitude']).reset_index(name='Count')

# Plot the density map
plt.figure(figsize=(10, 8))
plt.scatter(coor_burg['Latitude'], coor_burg['Longitude'], s=coor_burg['Count']*50, alpha=0.6)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density Map of Burglary Incidents in Hendon and west Henodn')
plt.show()

In [ ]:
# Uncomment the save code line to save the image
# Save the plot as an png file
# plt.savefig('Pictures/bus station analysis/DensityMapHendon.png')

In [ ]:
# Filter the LSOA data based on the specified names
filtered_lsoa = london_lsoa[london_lsoa['lsoa11nm'].isin(desired_LSOA)]

# Create a map centered around London
HeatMapwithBus = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Create a HeatMap layer for the density map
heat_data = [[row['Latitude'], row['Longitude'], row['Count']] for index, row in coor_burg.iterrows()]
HeatMap(heat_data).add_to(HeatMapwithBus)

# Iterate over each row in the dataframe
for index, row in bus_df.iterrows():
    # Get the easting and northing coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Check if the bus stop falls within the filtered LSOA boundaries
    point = Point(longitude, latitude)
    if filtered_lsoa.geometry.contains(point).any():
        # Create a marker for each bus stop
        bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
        bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                        popup=row['Stop_Name'],
                                        icon=bicycle_icon)
        
        # Add the marker to the map
        bus_stop_marker.add_to(HeatMapwithBus)

# Display the map
HeatMapwithBus

In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
# HeatMapwithBus.save('Pictures/Bus station analysis/HeatMapwithBus.html')

In [ ]:
# Filter the LSOA data based on the specified names
filtered_lsoa = london_lsoa[london_lsoa['lsoa11nm'].isin(desired_LSOA)]

# Read the London GeoJSON file
london_geojson = gpd.read_file('London.geojson')

# Filter the GeoJSON data to include only the desired LSOA boundaries
filtered_geojson = london_geojson[london_geojson['lsoa11nm'].isin(desired_LSOA)]

# Create a map centered around London
HeatMapWithBusLSOA = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Create a HeatMap layer for the density map
heat_data = [[row['Latitude'], row['Longitude'], row['Count']] for index, row in coor_burg.iterrows()]
HeatMap(heat_data).add_to(HeatMapWithBusLSOA)

# Add the LSOA boundaries to the map with black color
folium.GeoJson(filtered_geojson, style_function=lambda x: {'color': 'black'}).add_to(HeatMapWithBusLSOA)

# Iterate over each row in the dataframe
for index, row in bus_df.iterrows():
    # Get the easting and northing coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Check if the bus stop falls within the filtered LSOA boundaries
    point = Point(longitude, latitude)
    if filtered_lsoa.geometry.contains(point).any():
        # Create a marker for each bus stop
        bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
        bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                        popup=row['Stop_Name'],
                                        icon=bicycle_icon)
        
        # Add the marker to the map
        bus_stop_marker.add_to(HeatMapWithBusLSOA)

# Display the map
HeatMapWithBusLSOA


In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
# HeatMapWithBusLSOA.save('Pictures/Bus station analysis/HeatMapWithBusLSOA.html')

In [ ]:
# Filter the LSOA data based on the specified names
filtered_lsoa = london_lsoa[london_lsoa['lsoa11nm'].isin(desired_LSOA)]

# Create a map centered around London
BubbleMapWithBus = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Iterate over each row in the density data
for index, row in coor_burg.iterrows():
    # Get the latitude, longitude, and count
    latitude = row['Latitude']
    longitude = row['Longitude']
    count = row['Count']
    
    # Create a CircleMarker for each data point
    circle_marker = folium.CircleMarker(location=[latitude, longitude],
                                        radius=count,
                                        color='red',
                                        fill=True,
                                        fill_color='red',
                                        fill_opacity=0.6)
    
    # Add the CircleMarker to the map
    circle_marker.add_to(BubbleMapWithBus)

# Iterate over each row in the bus stop data
for index, row in bus_df.iterrows():
    # Get the easting and northing coordinates
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    # Check if the bus stop falls within the filtered LSOA boundaries
    point = Point(longitude, latitude)
    if filtered_lsoa.geometry.contains(point).any():
        # Create a marker for each bus stop
        bicycle_icon = folium.Icon(icon='bus', prefix='fa', color='green')
        bus_stop_marker = folium.Marker(location=[latitude, longitude],
                                        popup=row['Stop_Name'],
                                        icon=bicycle_icon)
        
        # Add the marker to the map
        bus_stop_marker.add_to(BubbleMapWithBus)

# Display the map
BubbleMapWithBus

In [ ]:
# Uncomment the save code line to save the image
# Save the map as an HTML file
# BubbleMapWithBus.save('Pictures/Bus station analysis/BubbleMapWithBus.html')

### The relation between the bus stop and their distance to the crime location

In [ ]:
barnet_crime_count = Barnet_df['LSOA name'].value_counts()
barnet_crime_count

In [ ]:
barnet_bus_count = Barnet_bus['LSOA name'].value_counts()
barnet_bus_count

In [ ]:
sum(barnet_bus_count)

In [ ]:
# Filter the crime count and bus count to include only the common LSOA names
common_lsoa_names = list(set(barnet_crime_count.index) & set(barnet_bus_count.index))
filtered_barnet_crime_count = barnet_crime_count[common_lsoa_names]
filtered_barnet_bus_count = barnet_bus_count[common_lsoa_names]

# Create a DataFrame with the filtered counts
df = pd.DataFrame({'Crime Count': filtered_barnet_crime_count, 'Bus Count': filtered_barnet_bus_count})

# Calculate the correlation between the counts
correlation = df['Crime Count'].corr(df['Bus Count'])

# Print the correlation
print("Correlation between Crime Count and Bus Count:", correlation)

In [ ]:
# Create a scatter plot of the crime count and bus count
plt.scatter(df['Crime Count'], df['Bus Count'])
plt.xlabel('Crime Count')
plt.ylabel('Bus Count')
plt.title('Correlation between Crime Count and Bus Count')
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
# Uncomment the save code line to save the image
# Save the plot as an png file
plt.savefig('correlation_plot.png')

### Calculating the Distance of each crime to the closest bus stop

In [ ]:
unique_location_combinations = len(Barnet_df.value_counts(['Longitude', 'Latitude']))
print("Number of unique combinations of longitude and latitude:", unique_location_combinations)

In [ ]:
count_threshold = 3
filtered_combinations = Barnet_df.value_counts(['Longitude', 'Latitude'])[Barnet_df.value_counts(['Longitude', 'Latitude']) >= count_threshold]
unique_filtered_combinations = len(filtered_combinations)
print("Number of unique combinations with count >= 3:", unique_filtered_combinations)

In [ ]:
count_threshold = 3
total_combinations = len(Barnet_df.value_counts(['Longitude', 'Latitude']))
filtered_combinations = len(Barnet_df.value_counts(['Longitude', 'Latitude'])[Barnet_df.value_counts(['Longitude', 'Latitude']) >= count_threshold])
rows_to_drop = total_combinations - filtered_combinations
print("Number of rows to drop:", rows_to_drop)

In [ ]:
count_threshold = 3
location_value_counts = Barnet_df.value_counts(['Longitude', 'Latitude'])
filtered_counts = location_value_counts[location_value_counts >= count_threshold]

# Create a larger figure
plt.figure(figsize=(15, 10))

# Create a distribution plot
sns.histplot(filtered_counts, kde=True)

# Set plot labels and title
plt.xlabel('Crime Count')
plt.ylabel('Frequency')
plt.title('Distribution of Crime counts for more than 3 crimes per location')

# Display the plot
plt.show()

In [ ]:
# Uncomment the save code line to save the image
# Save the plot as an png file
# plt.savefig('distribution_plot.png', dpi=300)

# checked till here

In [ ]:
filtered_counts = pd.DataFrame(filtered_counts).reset_index()
filtered_counts

In [ ]:
# Assuming filtered_counts is your DataFrame
filtered_counts['distance to bus'] = np.nan
filtered_counts

In [ ]:
Barnet_bus_location = Barnet_bus[['Longitude', 'Latitude']].reset_index()
Barnet_bus_location = Barnet_bus_location[['Longitude', 'Latitude']]
Barnet_bus_location

In [ ]:
filtered_counts = calculate_distance_to_nearest_bus(filtered_counts, Barnet_bus_location)
filtered_counts

In [ ]:
# Calculate the correlation between 'count' and 'distance to bus'
correlation = filtered_counts['count'].corr(filtered_counts['distance to bus'])
print(f"Correlation between 'count' and 'distance to bus': {correlation}")

# Create a scatter plot
plt.scatter(subset_filtered_counts['distance to bus'], subset_filtered_counts['count'])
plt.xlabel('Distance to Bus station')
plt.ylabel('Crime Count')
plt.title('Correlation between Count of Crimes and Distance to Bus stations')
plt.show()

In [ ]:
# Uncomment the save code line to save the image
# Save the plot as an png file
# plt.savefig('Pictures/Bus station analysis/CorrelationCrimeCountandDistancetoBus.png')

## season bus stops

In [ ]:
Barnet_df['Date month'] = pd.to_datetime(Barnet_df['Month']).dt.month

# Select values with the first 4 months of the year
selected_values = Barnet_df[Barnet_df['Date month'].isin([1, 2, 3, 4])]

# Print the selected values
selected_values['Month'].unique()

In [ ]:
# Assuming Barnet_df is your DataFrame containing the data
location_value_counts = selected_values.value_counts(['Longitude', 'Latitude'])
filtered_counts_season = location_value_counts[location_value_counts >= 0]

# Create a larger figure
plt.figure(figsize=(10, 6))

# Create a distribution plot
sns.histplot(filtered_counts_season, kde=True)

# Set plot labels and title
plt.xlabel('Count')
plt.ylabel('Frequency')
plt.title('Distribution of Counts')

# Display the plot
plt.show()

In [ ]:
filtered_counts_season = pd.DataFrame(filtered_counts_season).reset_index()
filtered_counts_season

In [ ]:
# Assuming filtered_counts is your DataFrame
filtered_counts_season['distance to bus'] = np.nan
filtered_counts_season

In [ ]:
Barnet_bus_location = Barnet_bus[['Longitude', 'Latitude']].reset_index()
Barnet_bus_location = Barnet_bus_location[['Longitude', 'Latitude']]
Barnet_bus_location

In [ ]:
calculate_distance_to_nearest_bus(filtered_counts_season, Barnet_bus_location)

In [ ]:
# Assuming subset_filtered_counts is your DataFrame with the 'count' and 'distance to bus' columns

# Calculate the correlation between 'count' and 'distance to bus'
correlation = subset_filtered_counts['count'].corr(subset_filtered_counts['distance to bus'])
print(f"Correlation between 'crime count' and 'distance to bus station' in first 4 months of the year: {correlation}")

## Number of crimes within X meters away from bus stops

In [ ]:
Barnet_bus_location = Barnet_bus[['Longitude', 'Latitude']].reset_index()
Barnet_bus_location = Barnet_bus_location[['Longitude', 'Latitude']]
Barnet_bus_location

In [ ]:
distance = 100000  # Specify the distance in meters
result_df, Nr_crimes = count_burglaries_within_distance(Barnet_bus_location, Barnet_df, distance)
result_df.to_csv(f'{distance} from busstop.csv')